In [1]:
import tensorflow as tf
from losses import TrainState
from flax.training import checkpoints
from flax import jax_utils as flax_utils
from matplotlib import pyplot as plt
import jax
from jax import numpy as jnp

from models import super_simple
from models import utils as mutils
import sampling
import losses

from configs.vp.disk_ssim_continuous import get_config
import sde_lib
import datasets

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from sklearn.datasets import make_swiss_roll
import os

2025-01-31 16:19:47.641121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%matplotlib qt

### Load State

In [7]:
rng = jax.random.PRNGKey(42)
model_rng, sampling_rng = jax.random.split(rng)
config = get_config()
score_model, init_model_state, init_model_params = mutils.init_model(model_rng, config)
optimizer = losses.get_optimizer(config)

state = TrainState.create(tx=optimizer, apply_fn=score_model.apply,
                          params=init_model_params, mutable_state=init_model_state,
                          rng=rng)
workdir = '/home/komodo/Documents/uni/thesis/score_sde'
ckpt_model_state = checkpoints.restore_checkpoint(os.path.join(workdir, 'checkpoints'), state)

### Setup SDE

In [8]:
if config.training.sde.lower() == 'vpsde':
    sde = sde_lib.VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
    sampling_eps = 1e-3
elif config.training.sde.lower() == 'subvpsde':
    sde = sde_lib.subVPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
    sampling_eps = 1e-3
elif config.training.sde.lower() == 'vesde':
    sde = sde_lib.VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=config.model.num_scales)
    sampling_eps = 1e-5
else:
    raise NotImplementedError(f"SDE {config.training.sde} unknown.")
inverse_scaler = datasets.get_data_inverse_scaler(config)

sampling_shape = (config.training.batch_size // jax.local_device_count(), config.data.image_size,
                config.data.image_size, config.data.num_channels)

### Sample Points
Samples 16 * batch_size points for every time step (50 time steps)

In [9]:
num_rng = 16
num_sample_steps = 50

# sampling_fn = pc_sampler
sampling_fn = sampling.get_sampling_fn(config, sde, score_model, sampling_shape, inverse_scaler, sampling_eps)
pstate = flax_utils.replicate(ckpt_model_state)
num_train_steps = config.training.n_iters

# In case there are multiple hosts (e.g., TPU pods), only log to host 0
rng = jax.random.fold_in(rng, jax.host_id())
rng, *next_rng = jax.random.split(rng, num=jax.local_device_count() + 1)
next_rng = jnp.asarray(next_rng)

rng_vmap = jax.vmap(lambda r, t: sampling_fn(r, pstate, jnp.array([t])), in_axes=(0, None), out_axes=0)
both_vmap = jax.vmap(lambda r, t: rng_vmap(r, t), in_axes=(None, 0), out_axes=1)

rng, *next_rng = jax.random.split(rng, num=jax.local_device_count() * num_rng + 1)
next_rng = jnp.asarray(next_rng)[:, None, :]
next_rng.shape

ts = jnp.floor(jnp.linspace(0, sde.N, num_sample_steps)).astype(jnp.int32)

a, b = both_vmap(next_rng, ts)
a_sq = a.reshape((num_rng, num_sample_steps, config.training.batch_size, a.shape[-1]))
temp = a_sq.transpose((1, 0, 2, 3)).reshape((num_sample_steps, -1, a.shape[-1]))
xs = temp[:, :, 0]
ys = temp[:, :, 2 if config.data.dataset == 'SWIRL' else 1]
jnp.mean(jnp.sqrt(jnp.sum(temp**2, axis=-1)), axis=1)

/home/komodo/miniconda3/envs/testing/lib/python3.11/site-packages/jax/_src/xla_bridge.py:1174: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


Array([1.227253  , 1.1660728 , 1.2094907 , 1.2264435 , 1.2414682 ,
       1.2663429 , 1.2801079 , 1.3083363 , 1.2962086 , 1.2943906 ,
       1.2903306 , 1.2917602 , 1.2903299 , 1.2884624 , 1.275465  ,
       1.2746811 , 1.2756102 , 1.2773843 , 1.2800659 , 1.2798281 ,
       1.2663257 , 1.2500153 , 1.2455401 , 1.228946  , 1.206138  ,
       1.1938279 , 1.1806648 , 1.1646523 , 1.1478417 , 1.1335125 ,
       1.1248211 , 1.0978451 , 1.079571  , 1.0460877 , 1.0107687 ,
       0.98613226, 0.97160274, 0.9381471 , 0.90366703, 0.8722153 ,
       0.8495688 , 0.81314015, 0.7814474 , 0.748702  , 0.724242  ,
       0.7056156 , 0.6898152 , 0.6772318 , 0.66648537, 0.66265583],      dtype=float32)

### Visualise Data

Prepare ground truth

In [30]:
_, test_ds, _ = datasets.get_dataset(config)
gt = tf.stack([x['image'] for x in test_ds.take(num_rng)]).numpy().reshape(temp.shape[1:])

Visualise ground truth

In [32]:
fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(5)

circle = plt.Circle((0, 0), 1, color='r', fill=False)

ax.add_patch(circle)
line, = ax.plot(gt[:, 0], gt[:, 1], 'r+')

ax.grid(True)

plt.show()

Visualise Data

In [34]:
fig, ax = plt.subplots()
fig.set_figheight(5)
fig.set_figwidth(5)

circle = plt.Circle((0, 0), 1, color='r', fill=False)

ax.add_patch(circle)
line, = ax.plot(xs[-1], ys[-1], 'r+')

axamp = fig.add_axes([0.01, 0.2, 0.0225, 0.63])
t_slider = Slider(
    ax=axamp,
    label="T",
    valmin=0,
    valmax=sde.N,
    valinit=sde.N,
    orientation="vertical",
    valstep=ts
)

# The function to be called anytime a slider's value changes
def update(val):
    ind = jnp.where(ts == val)[0]
    line.set_xdata(xs[ind])
    line.set_ydata(ys[ind])
    fig.canvas.draw_idle()


# register the update function with each slider
t_slider.on_changed(update)

ax.grid(True)

plt.show()